In this tutorial, we will demonstrate how to implement a time-dependent simulation of hydrogen isotopes diffusion-trapping followed by Thermal Desorption Spectrometry (TDS). The aim is to replicate the experimental results presented in [Ogorodnikova _et al_ (J. Nucl. Mater. 313-316 (2003) 469-477.)](https://www.sciencedirect.com/science/article/pii/S0022311502013752?via%3Dihub) using FESTIM.

First import `FESTIM` and `FESTIM.generic_simulation` modules

In [1]:
from FESTIM import *
from FESTIM.generic_simulation import *

# 1. Setting the parameters
The function `run()` in FESTIM takes as an argument a dictionary (dict) of parameters. The first step is to create an empty dict.  The parameters needed for the simulation are added in the following steps.

In [2]:
parameters = {}

### 1.1 Mesh


FESTIM provides a simple way to define refined meshes. 
The size of the 1D domain we wish to mesh is set in `"size"` and is expressed in $\text{m}$. In our case, we simulate a $20 \mu\text{m}$ slab. We start with a broad mesh of 200 cells and then iteratively refine it.
The following code performs a first refinement within the first $3 \mu\text{m}$ in order to have at least 300 cells in this region. We will do the same for the first $30 \text{nm}$ where we want at least 120 cells.

In [3]:
mesh_parameters = {
            "size": 20e-6,
            "initial_number_of_cells": 200,
            "refinements": [
                {
                    "cells": 300,
                    "x": 3e-6
                },
                {
                    "cells": 120,
                    "x": 30e-9
                }
            ],
        }
parameters["mesh_parameters"] = mesh_parameters

### 1.2 Material properties
The materials are defined as a list of dict. In our case, we have only one material (tungsten), the list will therefore have only one element.

 `"borders"` represents the boundaries of the 1D domain in $\text{m}$.
 
 `"E_D"` is the energy barrier for diffusion expressed in $\text{eV}$.
 
 `"D_0"` is the pre-exponential factor of the Arhenius' law for the diffusion coefficient expressed in $\text{m}^{2}\text{s}^{-1}$.
 Diffusion coefficient is expressed as:
\begin{equation}
    D(T) = D_{0} e^{\frac{-E_{diff}}{k_B \cdot T}}
\end{equation}

where $T$ is the temperature in $\text{K}$ and $k_B = 8.6\times 10 ^{-5} \: \text{eV/K}$ the Boltzmann constant.

Finally, `"id"` is the id of the material subdomain. In this case, there is only one.

In [4]:
material = [{
            "borders": [0, 20e-6],
            "E_D": 0.39,
            "D_0": 4.1e-7,
            "id": 1
}]
parameters["materials"] = material

### 1.3 Volumetric source term

We define next the volumetric source term for the simulation as a dict.

First we import the sympy package needed to handle the source term as a sympy object in FESTIM.

The source term is defined as:
\begin{equation}
    S_{ext} = \varphi \cdot f(x) \quad \forall t<400 \text{s}
\end{equation}

where $\varphi =2.5 \times 10^{19} \text{m}^{-2}\text{s}^{-1}$ and $f(x)$ is a Gaussian spatial distribution with a mean value of $4.5 \text{nm}$ and a width of $2.5 \text{nm}$.

Below, `t` and `x` are built-in variables that respectively represent time in $\text{s}$ and the $x$ position in $\text{m}$.

In [5]:
import sympy as sp


center = 4.5e-9
width = 2.5e-9
distribution = 1/(width*(2*3.14)**0.5) * sp.exp(-0.5*((x-center)/width)**2)

source_term = {
        'value': 2.5e19 *distribution* (t <= 400)
        }
parameters["source_term"] = source_term

### 1.4 Traps

In FESTIM, each trap is defined by a dictionary with the following keys:

 `"k_0"` and `"E_k"` are the trapping rate pre-exponential factor in $\text{m}^{3} \text{s}^{-1}$ and activation energy in $\text{eV}$.

 `"p_0"` and `"E_p"` are the detrapping rate pre-exponential factor in $\text{s}^{-1}$ and activation energy in $\text{eV}$. 

 `"density"` is the trap density in $\text{m}^{-3}$. 
 
 `"materials"` is a list of the materials' ids where the trap is. 
 
`"type"` is the trap type. 

 In FESTIM, traps are intrinsic by default. However, traps can also be implemented as extrinsic, as shown in the example hereafter corresponding to 2 intrinsic traps and 1 extrinsic trap.

The time evolution of extrinsic traps density $n_i$ expressed in $\text{m}^{-3}$ is defined as:
 \begin{equation}
    \frac{dn_i}{dt} = \varphi_0\:[(1-\frac{n_i}{n_{a_{max}}})\:\eta_a \:f_a(x)+(1-\frac{n_i}{n_{b_{max}}})\:\eta_b \:f_b(x)]
 \end{equation}
 
 

In [ ]:
traps = [
        {
            "k_0": 4.1e-7/(1.1e-10**2*6*6.3e28)
            "E_k": 0.39,
            "p_0": 1e13,
            "E_p": 0.87,
            "density": 1.3e-3*6.3e28,
            "materials": [1]
        },
        {
            "k_0": 4.1e-7/(1.1e-10**2*6*6.3e28)
            "E_k": 0.39,
            "p_0": 1e13,
            "E_p": 1.0,
            "density": 4e-4*6.3e28,
            "materials": [1]
        },
        {
            "k_0": 4.1e-7/(1.1e-10**2*6*6.3e28)
            "E_k": 0.39,
            "p_0": 1e13,
            "E_p": 1.5,
            "materials": [1],
            "type": 'extrinsic',
            "form_parameters":{
                "phi_0": 2.5e19* (t <= 400),
                "n_amax": 1e-1*6.3e28,
                "f_a": distribution,
                "eta_a": 6e-4,
                "n_bmax": 1e-2*6.3e28,
                "f_b": (x < 1e-6) * (x > 0) * (1/1e-6),
                "eta_b": 2e-4,
            }
        }
        ]

parameters["traps"] = traps

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed.

### 1.5 Boundary conditions

Boundary conditions (BCs) are defined as a list of dicts. BCs can be of several types in FESTIM, the most simple of them being the `"dc"` type where an analytical expression is given in `"value"`. The key `"surface"` contains a list of all the surfaces on which the BC is applied. If no BC is applied on a surface, it will be considered as a non flux surface (ie $\frac{\partial c}{\partial\textbf{n}} = 0$)

**Note** : `"component"` is an optional parameter and is by default 0. It refers to the component of the solution vector, with 0 being the solute population, and $i$ the trap $i$ population.

In [7]:
boundary_conditions = [
            {
                "surfaces": [1],
                "value": 0,
                "component": 0,
                "type": "dc"
            },
            {
                "surfaces": [2],
                "value": 0,
                "type": "dc"
            }
    ]

parameters["boundary_conditions"] = boundary_conditions

### 1.6 Temperature

Temperature has a major effect on HIs' behavior. It is once again defined as a dict. Temperature can be of several types (constant, expression) in FESTIM. 

In this example, temperature is constant from $t=0$ to $t=450 \text{s}$ (implantation phase), then increases from $t=450 \text{s}$ to $t=500 \text{s}$ in order to perform the thermo-desorption (TDS phase).

\begin{equation}
T(t) = 
    \begin{cases}
    300, & \text{if} \: t < 450 \\
    300 + 8(t - 450), & \text{else}
    \end{cases}
\end{equation}

$T$ is expressed in $\text{K}$.

In [8]:
temperature = {
            "type": "expression",
            "value": 300 + (t > 450) * (8*(t-450))
        }

parameters["temperature"] = temperature

### 1.7 Solver parameters

The first parameter is the final time of the simulation. As seen before, this run corresponds to a 450s implantation phase followed by 50s of TDS, which corresponds to a $500 \text{s}$ final time.

Then the initial stepsize is expressed in $\text{s}$ in `"initial_stepsize"`.

An adaptive stepsize algorithm has been implemented in order to save computational cost depending on the last timestep. The parameters of the algorithm are set in `"adaptative_time_step"`. `"t_stop"` is the time when we want the stepsize to be below a value set in `"stepsize_stop_max"` in order to have temporal refinement at the end of the run. `"dt_min"` is the lower limit for the stepsize below which the computation will stop.

**Note**: a stepsize change ratio of 1 will disable the adaptive stepsize algorithm.

The Newton solver used in FEniCS can be tweaked by setting the absolute and relative tolerances as well as the maximum number of iteration at each step.

In [9]:
solving_parameters = {
        "final_time": 500,
        "initial_stepsize": 0.5,
        "adaptive_stepsize": {
            "stepsize_change_ratio": 1.1,
            "t_stop": 430,
            "stepsize_stop_max": 0.5,
            "dt_min": 1e-5
            },
        "newton_solver": {
            "absolute_tolerance": 1e10,
            "relative_tolerance": 1e-9,
            "maximum_iterations": 50,
        }
        }

parameters["solving_parameters"] = solving_parameters

### 1.8 Exports

The quantities exported by FESTIM are defined using a dict of dicts.

1D profiles can be exported to a .txt file at specific times and/or exported to XDMF files at each timestep. In both cases, the fields to be exported are defined in `"functions"` as a list of strings. 

`"solute"` refers to the mobile concentration of HIs, $i$ is the concentration of HIs trapped in trap $i$ and `"retention"` is the total ammount of HIs. `"labels"` will be the name of the exports files and can be anything we want.

The desorption spectrum (`"TDS"`) can also be exported to a .csv file. The starting time of the desorption (ie. time from which desorption is calculated) is defined in `"TDS_time"`.

In the example below, all exports will be stored in the folder Solution_Ogorodnikova/.

In [10]:
folder = 'Solution_Ogorodnikova'

exports = {
        "txt": {
            "functions": ['retention'],
            "times": [100, 200],
            "labels": ['retention'],
            "folder": folder
        },
        "xdmf": {
            "functions": ['solute', '1', '2', '3', 'retention'],
            "labels":  ['solute', 'trap_1', 'trap_2',
                        'trap_3', 'retention'],
            "folder": folder
        }
        }

parameters["exports"] = exports


In the end, all the parameters needed for the simulation in FESTIM have been stored in the dict `parameters`, which we can preview for a final check:

In [11]:
for key, value in parameters.items():
    print(str(key) + ":" + str(value))
    print('\n')

mesh_parameters:{'size': 2e-05, 'initial_number_of_cells': 200, 'refinements': [{'cells': 300, 'x': 3e-06}, {'cells': 120, 'x': 3e-08}]}


materials:[{'alpha': 1.1e-10, 'beta': 3.7800000000000004e+29, 'density': 6.3e+28, 'borders': [0, 2e-05], 'E_diff': 0.39, 'D_0': 4.1e-07, 'id': 1}]


source_term:{'value': 3.99043442233811e+27*exp(-0.5*(400000000.0*x[0] - 1.8)**2)*(t <= 400)}


traps:[{'energy': 0.87, 'density': 8.19e+25, 'materials': [1]}, {'energy': 1.0, 'density': 2.52e+25, 'materials': [1]}, {'energy': 1.5, 'materials': [1], 'type': 'extrinsic', 'form_parameters': {'phi_0': 2.5e+19*(t <= 400), 'n_amax': 6.3e+27, 'f_a': 159617376.893524*exp(-0.5*(400000000.0*x[0] - 1.8)**2), 'eta_a': 0.0006, 'n_bmax': 6.3e+26, 'f_b': 1000000.0*(x[0] < 1.0e-6)*(x[0] > 0), 'eta_b': 0.0002}}]


boundary_conditions:[{'surface': [1], 'value': 0, 'component': 0, 'type': 'dc'}, {'surface': [2], 'value': 0, 'type': 'dc'}]


temperature:{'type': 'expression', 'value': (8*t - 3600)*(t > 450) + 300}


solvin

# 2. Running the simulation

Now that all the parameters have been set, the simulation can be performed using the `run()` function which takes as only argument the dict `parameters`.

In [12]:
output = run(parameters)

Meshing ...
Mesh size before local refinement is 200
Mesh size after local refinement is 650
Mesh size before local refinement is 650
Mesh size after local refinement is 798
Defining source terms
Defining initial values
Defining variational problem
Defining boundary conditions
Time stepping...
s0.01 %        500.0 s    Ellapsed time so far: 43.1 s


# 3. Output

Now that the simulation is over, several objects such as the parameters dict, the mesh, the temperature and the desorption table  are stored in `output`.

Let us see what `output` contains:

In [13]:
print(output.keys())

dict_keys(['parameters', 'mesh', 'temperature', 'TDS'])


`output["temperature"]` contains the temperature at every timestep:

In [14]:
import numpy as np
np.set_printoptions(threshold=20)
print(np.array(output["temperature"]))

[['t (s)' 'T (K)']
 ['0.5' '300.0']
 ['0.9545454545454546' '300.0']
 ..., 
 ['499.0266404933807' '692.213123947']
 ['499.5266404933807' '696.213123947']
 ['500.0266404933807' '700.213123947']]


`output["TDS"]` corresponds to the desorption data that has also been exported to the .csv file. 

In [15]:
print(np.array(output["TDS"]))

[['t (s)' 'T (K)' 'd (m-2.s-1)']
 ['450.0266404933807' '300.213123947' '2.315922518618931e+16']
 ['450.5266404933807' '304.213123947' '3.5708037039128576e+16']
 ..., 
 ['499.0266404933807' '692.213123947' '1.5153242315658035e+17']
 ['499.5266404933807' '696.213123947' '1.2128713680071763e+17']
 ['500.0266404933807' '700.213123947' '9.455067303297434e+16']]


The desorption spectra (ie. desorption flux as function of temperature) can be plotted, as shown below:

![title](docs/TDS_Ogorodnikova.png)